<a href="https://colab.research.google.com/github/LuisMHS/Python-y-ESP32WROVER/blob/main/Interfaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import flet as ft
import cv2
import time
import base64
from threading import Thread
import os

class CameraStream:
    def __init__(self, url):
        self.cap = cv2.VideoCapture(url)
        self.running = True
        self.frame = None
        self.recording = False
        self.video_writer = None

    def start(self):
        def run():
            while self.running:
                ret, frame = self.cap.read()
                if ret:
                    self.frame = frame
                    if self.recording and self.video_writer:
                        self.video_writer.write(frame)
                time.sleep(0.03)
        Thread(target=run, daemon=True).start()

    def get_frame_base64(self):
        if self.frame is None:
            return None
        _, buffer = cv2.imencode('.jpg', self.frame)
        return base64.b64encode(buffer).decode('utf-8')

    def take_photo(self):
        if self.frame is not None:
            filename = f"captures/photo_{int(time.time())}.jpg"
            cv2.imwrite(filename, self.frame)
            return filename
        return None

    def start_recording(self):
        if self.frame is not None:
            h, w = self.frame.shape[:2]
            filename = f"captures/video_{int(time.time())}.avi"
            self.video_writer = cv2.VideoWriter(
                filename, cv2.VideoWriter_fourcc(*'XVID'), 20, (w, h)
            )
            self.recording = True
            return filename
        return None

    def stop_recording(self):
        self.recording = False
        if self.video_writer:
            self.video_writer.release()
            self.video_writer = None

    def stop(self):
        self.running = False
        self.cap.release()
        self.stop_recording()


def main(page: ft.Page):
    page.title = "Cámara ESP32 + Grabación"
    page.scroll = "auto"

    img = ft.Image(src="", width=640, height=480)
    status = ft.Text("Estado: Conectado")

    cam = CameraStream("http://192.168.1.6:81/stream")
    cam.start()

    def update_image():
        while True:
            frame_data = cam.get_frame_base64()
            if frame_data:
                img.src_base64 = frame_data
                page.update()
            time.sleep(0.05)

    def take_photo(e):
        filename = cam.take_photo()
        if filename:
            status.value = f"Foto guardada: {filename}"
            page.update()

    def start_recording(e):
        filename = cam.start_recording()
        if filename:
            status.value = f"Grabando: {filename}"
            page.update()

    def stop_recording(e):
        cam.stop_recording()
        status.value = "Grabación detenida"
        page.update()

    # Botones
    btn_foto = ft.ElevatedButton("📸 Tomar foto", on_click=take_photo)
    btn_grabar = ft.ElevatedButton("🎥 Iniciar grabación", on_click=start_recording)
    btn_parar = ft.ElevatedButton("⏹️ Detener grabación", on_click=stop_recording)

    page.add(img, ft.Row([btn_foto, btn_grabar, btn_parar]), status)
    Thread(target=update_image, daemon=True).start()

ft.app(target=main, view=None,port=8550,host="0.0.0.0")

